In [1]:
file_list_GS36_MPC001=open("/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_GS_y2012-2013/GS36-MPC001/list_vert_GS36-MPC001.txt","r")
list_GS36_MPC001=file_list_GS36_MPC001.read()
list_list_GS_MPC001=list_GS36_MPC001.split("\n")

In [2]:
file_list_GS36_MPC002=open("/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_GS_y2012-2013/GS36-MPC002/list_vert_GS36-MPC002.txt","r")
list_GS36_MPC002=file_list_GS36_MPC002.read()
prof_GS_MPC002=list_GS36_MPC002.split("\n")

In [3]:
list_commune_MPC001_MPC002=[]
for prof in list_list_GS_MPC001:
    if prof in prof_GS_MPC002:
        list_commune_MPC001_MPC002.append(prof)
        
    

In [4]:
file_list_GS36_MPC003=open("/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_GS_y2012-2013/GS36-MPC003/list_vert_GS36-MPC003.txt","r")
list_GS36_MPC003=file_list_GS36_MPC003.read()
prof_GS_MPC003=list_GS36_MPC003.split("\n")

In [5]:
list_commune_MPC001_MPC002_MPC003=[]
for prof in list_commune_MPC001_MPC002:
    if prof in prof_GS_MPC003:
        list_commune_MPC001_MPC002_MPC003.append(prof)
        
    

In [6]:
print(len(list_commune_MPC001_MPC002_MPC003))

18174


In [7]:
list_prof_commune_MPC001_MPC002_MPC003=[]
for prof in list_commune_MPC001_MPC002_MPC003:
    list_prof_commune_MPC001_MPC002_MPC003.append(prof[4:])


In [8]:
print(str(list_prof_commune_MPC001_MPC002_MPC003[0]))

A20121001-03173


In [9]:
import numpy as np
import dask
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import time
from datetime import date
import io
import json


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [10]:
latmin,latmax,lonmin,lonmax=(27,48,-81,-40)
datemin,datemax=('2012-10-01','2013-09-30')
jsonfile='NATL60-CJM165_GS_y2012-2013_final.json'

In [11]:
file='/scratch/cnt0024/hmg2840/albert7a/EN4/EN.4.2.0.f.profiles.g10.201201.nc'
ds=xr.open_dataset(file)

In [12]:
def select_profiles(latmin,latmax,lonmin,lonmax,datemin,datemax,jsonfile):
        ''' Identify all the EN4 profiles that fall within the selected zone and period
        '''
        ## Datasets

        #EN4
        diren4="/scratch/cnt0024/hmg2840/albert7a/EN4/"

        yearmin=datemin[0:4]
        monthmin=datemin[5:7]
        yearmax=datemax[0:4]
        monthmax=datemax[5:7]

        list_filesEN4=[]
        if yearmin == yearmax:
            for m in np.arange(int(monthmin),int(monthmax)+1):
                if m < 10:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+'0'+str(m)+'.nc')
                else:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+str(m)+'.nc')
        else:
            for m in np.arange(int(monthmin),13):
                if m < 10:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+'0'+str(m)+'.nc')
                else:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+str(m)+'.nc')
        if int(yearmin)+1 == int(yearmax):
            for m in np.arange(1,int(monthmax)+1):
                if m < 10:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmax+'0'+str(m)+'.nc')
                else:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmax+str(m)+'.nc')
        else:
            for year in np.arange(int(yearmin)+1,int(yearmax)):
                for m in np.arange(1,13):
                    if m < 10:
                        list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+str(year)+'0'+str(m)+'.nc')
                    else:
                        list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+str(year)+str(m)+'.nc')
        print(list_filesEN4)

        datetmin=pd.to_datetime(datemin)
        datetmax=pd.to_datetime(datemax)
        ttmin=datetime.datetime(int(datetmin.strftime('%Y')),int(datetmin.strftime('%m')),int(datetmin.strftime('%d')),int(datetmin.strftime('%H')),int(datetmin.strftime('%M')))
        ttmax=datetime.datetime(int(datetmax.strftime('%Y')),int(datetmax.strftime('%m')),int(datetmax.strftime('%d')),int(datetmax.strftime('%H')),int(datetmax.strftime('%M')))
        tsecmin=(ttmin-datetime.datetime(1958,1,1,0,0)).total_seconds()
        tsecmax=(ttmax-datetime.datetime(1958,1,1,0,0)).total_seconds()

        for f in np.arange(len(list_filesEN4)):
            fileEN4=list_filesEN4[f]
            print('Processing '+str(fileEN4))
            tfileEN4=diren4+fileEN4
            dsen4=xr.open_dataset(tfileEN4)
            laten4=dsen4['LATITUDE']
            lonen4=dsen4['LONGITUDE']
            dayen4=dsen4['JULD']
            refen4=dsen4['DC_REFERENCE']
                      
            indz=np.where((lonmin<lonen4.values)&(lonen4.values<lonmax)&(latmin<laten4.values)&(laten4.values<latmax))
            prof_zone=[]
            tsecen4z=[]
            for ref in np.arange(len(indz[0])):
                dateen4= pd.to_datetime(str(dayen4[indz[0][ref]].values))
                ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
                tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()
                if (tsecen4 < tsecmax) & (tsecen4 > tsecmin):
                    prof_zone.append(indz[0][ref])
                    tsecen4z.append(tsecen4)
                    

            for ref in np.arange(len(prof_zone)):
                ref_str=str(refen4[prof_zone[ref]].values)[-16:-1]
                if ref_str in list_prof_commune_MPC001_MPC002_MPC003:
                    print(ref_str+' est bien dans la liste')
                    if 'dictyml' in locals():
                        up={str(refen4[prof_zone[ref]].values):{'reference':str(refen4[prof_zone[ref]].values),'file':fileEN4,'profile no':int(prof_zone[ref]),'latitude':float(laten4[prof_zone[ref]].values),'longitude':float(lonen4[prof_zone[ref]].values),'date':str(dayen4[prof_zone[ref]].values)}}
                        dictyml.update(up)
                    else:
                        dictyml={str(refen4[prof_zone[ref]].values):{'reference':str(refen4[prof_zone[ref]].values),'file':fileEN4,'profile no':int(prof_zone[ref]),'latitude':float(laten4[prof_zone[ref]].values),'longitude':float(lonen4[prof_zone[ref]].values),'date':str(dayen4[prof_zone[ref]].values)}}
                    
        with io.open(jsonfile, 'w', encoding='utf8') as outfile:
            outfile.write(str(json.dumps(dictyml, sort_keys=True,indent=4, separators=(',', ': '))))



In [13]:
select_profiles(latmin,latmax,lonmin,lonmax,datemin,datemax,jsonfile)

['EN.4.2.0.f.profiles.g10.201210.nc', 'EN.4.2.0.f.profiles.g10.201211.nc', 'EN.4.2.0.f.profiles.g10.201212.nc', 'EN.4.2.0.f.profiles.g10.201301.nc', 'EN.4.2.0.f.profiles.g10.201302.nc', 'EN.4.2.0.f.profiles.g10.201303.nc', 'EN.4.2.0.f.profiles.g10.201304.nc', 'EN.4.2.0.f.profiles.g10.201305.nc', 'EN.4.2.0.f.profiles.g10.201306.nc', 'EN.4.2.0.f.profiles.g10.201307.nc', 'EN.4.2.0.f.profiles.g10.201308.nc', 'EN.4.2.0.f.profiles.g10.201309.nc']
Processing EN.4.2.0.f.profiles.g10.201210.nc
A20121001-03173 est bien dans la liste
A20121001-03175 est bien dans la liste
A20121001-03182 est bien dans la liste
A20121001-03196 est bien dans la liste
A20121001-03200 est bien dans la liste
A20121001-03206 est bien dans la liste
A20121001-03208 est bien dans la liste
A20121001-03213 est bien dans la liste
A20121001-03215 est bien dans la liste
A20121001-03227 est bien dans la liste
A20121001-03234 est bien dans la liste
A20121001-03236 est bien dans la liste
A20121001-03247 est bien dans la liste
A20

A20121018-05065 est bien dans la liste
A20121018-05082 est bien dans la liste
A20121018-05096 est bien dans la liste
A20121019-05105 est bien dans la liste
A20121019-05148 est bien dans la liste
A20121019-05156 est bien dans la liste
A20121019-05160 est bien dans la liste
A20121019-05162 est bien dans la liste
A20121019-05169 est bien dans la liste
A20121019-05186 est bien dans la liste
A20121019-05197 est bien dans la liste
A20121020-05210 est bien dans la liste
A20121020-05212 est bien dans la liste
A20121020-05214 est bien dans la liste
A20121020-05242 est bien dans la liste
A20121020-05256 est bien dans la liste
A20121020-05260 est bien dans la liste
A20121020-05272 est bien dans la liste
A20121020-05293 est bien dans la liste
A20121021-05312 est bien dans la liste
A20121021-05318 est bien dans la liste
A20121021-05334 est bien dans la liste
A20121021-05349 est bien dans la liste
A20121021-05358 est bien dans la liste
A20121021-05361 est bien dans la liste
A20121021-05363 est bien 

A20121031-39540 est bien dans la liste
A20121031-39541 est bien dans la liste
A20121031-39542 est bien dans la liste
A20121031-39543 est bien dans la liste
A20121031-39544 est bien dans la liste
A20121031-39545 est bien dans la liste
A20121031-39546 est bien dans la liste
A20121031-39547 est bien dans la liste
A20121031-39548 est bien dans la liste
A20121031-39549 est bien dans la liste
A20121031-39550 est bien dans la liste
A20121031-39551 est bien dans la liste
A20121031-39718 est bien dans la liste
A20121031-39719 est bien dans la liste
A20121030-39720 est bien dans la liste
A20121001-39744 est bien dans la liste
A20121001-39745 est bien dans la liste
A20121001-39746 est bien dans la liste
A20121001-39747 est bien dans la liste
A20121001-39748 est bien dans la liste
A20121001-39756 est bien dans la liste
A20121001-39757 est bien dans la liste
A20121001-39758 est bien dans la liste
A20121001-39759 est bien dans la liste
A20121001-39760 est bien dans la liste
A20121001-40026 est bien 

A20121012-43004 est bien dans la liste
A20121012-43005 est bien dans la liste
A20121012-43006 est bien dans la liste
A20121010-43007 est bien dans la liste
A20121010-43008 est bien dans la liste
A20121010-43009 est bien dans la liste
A20121010-43010 est bien dans la liste
A20121010-43011 est bien dans la liste
A20121010-43012 est bien dans la liste
A20121010-43013 est bien dans la liste
A20121011-43014 est bien dans la liste
A20121011-43015 est bien dans la liste
A20121011-43016 est bien dans la liste
A20121011-43017 est bien dans la liste
A20121011-43018 est bien dans la liste
A20121011-43019 est bien dans la liste
A20121011-43020 est bien dans la liste
A20121011-43021 est bien dans la liste
A20121011-43022 est bien dans la liste
A20121011-43023 est bien dans la liste
A20121011-43024 est bien dans la liste
A20121012-43025 est bien dans la liste
A20121012-43026 est bien dans la liste
A20121012-43027 est bien dans la liste
A20121012-43028 est bien dans la liste
A20121012-43029 est bien 

A20121021-45966 est bien dans la liste
A20121021-45967 est bien dans la liste
A20121021-45968 est bien dans la liste
A20121021-45969 est bien dans la liste
A20121021-45970 est bien dans la liste
A20121022-45971 est bien dans la liste
A20121022-45972 est bien dans la liste
A20121022-45973 est bien dans la liste
A20121022-45974 est bien dans la liste
A20121022-45975 est bien dans la liste
A20121022-45976 est bien dans la liste
A20121022-46620 est bien dans la liste
A20121022-46621 est bien dans la liste
A20121022-46622 est bien dans la liste
A20121022-46623 est bien dans la liste
A20121022-46624 est bien dans la liste
A20121022-46625 est bien dans la liste
A20121023-46626 est bien dans la liste
A20121023-46627 est bien dans la liste
A20121023-46628 est bien dans la liste
A20121023-46629 est bien dans la liste
A20121023-46630 est bien dans la liste
A20121023-46631 est bien dans la liste
A20121023-46632 est bien dans la liste
A20121023-46633 est bien dans la liste
A20121023-46634 est bien 

...

A20130922-79192 est bien dans la liste
A20130922-79194 est bien dans la liste
A20130922-79195 est bien dans la liste
A20130922-79225 est bien dans la liste
A20130922-79226 est bien dans la liste
A20130922-79227 est bien dans la liste
A20130922-79228 est bien dans la liste
A20130922-79229 est bien dans la liste
A20130922-79230 est bien dans la liste
A20130922-79231 est bien dans la liste
A20130922-79232 est bien dans la liste
A20130922-79233 est bien dans la liste
A20130922-79234 est bien dans la liste
A20130922-79237 est bien dans la liste
A20130922-79238 est bien dans la liste
A20130922-79239 est bien dans la liste
A20130922-79277 est bien dans la liste
A20130922-79278 est bien dans la liste
A20130922-79280 est bien dans la liste
A20130922-79281 est bien dans la liste
A20130922-79282 est bien dans la liste
A20130922-79283 est bien dans la liste
A20130922-79284 est bien dans la liste
A20130922-79286 est bien dans la liste
A20130922-79287 est bien dans la liste
A20130922-79288 est bien 

A20130923-80131 est bien dans la liste
A20130923-80132 est bien dans la liste
A20130923-80133 est bien dans la liste
A20130923-80134 est bien dans la liste
A20130923-80135 est bien dans la liste
A20130923-80136 est bien dans la liste
A20130923-80137 est bien dans la liste
A20130923-80138 est bien dans la liste
A20130923-80143 est bien dans la liste
A20130923-80144 est bien dans la liste
A20130923-80181 est bien dans la liste
A20130923-80182 est bien dans la liste
A20130923-80183 est bien dans la liste
A20130923-80184 est bien dans la liste
A20130923-80185 est bien dans la liste
A20130923-80186 est bien dans la liste
A20130923-80187 est bien dans la liste
A20130923-80188 est bien dans la liste
A20130923-80189 est bien dans la liste
A20130923-80191 est bien dans la liste
A20130923-80192 est bien dans la liste
A20130923-80193 est bien dans la liste
A20130923-80194 est bien dans la liste
A20130923-80195 est bien dans la liste
A20130923-80218 est bien dans la liste
A20130923-80219 est bien 

A20130924-80868 est bien dans la liste
A20130924-80869 est bien dans la liste
A20130924-80870 est bien dans la liste
A20130924-80871 est bien dans la liste
A20130924-80872 est bien dans la liste
A20130924-80873 est bien dans la liste
A20130924-80918 est bien dans la liste
A20130924-80919 est bien dans la liste
A20130924-80920 est bien dans la liste
A20130924-80921 est bien dans la liste
A20130924-80922 est bien dans la liste
A20130924-80923 est bien dans la liste
A20130924-80924 est bien dans la liste
A20130924-80925 est bien dans la liste
A20130924-80928 est bien dans la liste
A20130924-80929 est bien dans la liste
A20130924-80930 est bien dans la liste
A20130924-80931 est bien dans la liste
A20130924-80932 est bien dans la liste
A20130924-80967 est bien dans la liste
A20130924-80968 est bien dans la liste
A20130924-80969 est bien dans la liste
A20130924-80970 est bien dans la liste
A20130924-80971 est bien dans la liste
A20130924-80972 est bien dans la liste
A20130924-80973 est bien 

A20130925-81842 est bien dans la liste
A20130925-81843 est bien dans la liste
A20130925-81896 est bien dans la liste
A20130925-81897 est bien dans la liste
A20130925-81900 est bien dans la liste
A20130925-81901 est bien dans la liste
A20130925-81902 est bien dans la liste
A20130925-81903 est bien dans la liste
A20130925-81961 est bien dans la liste
A20130925-81962 est bien dans la liste
A20130925-81968 est bien dans la liste
A20130925-81969 est bien dans la liste
A20130925-81970 est bien dans la liste
A20130925-81971 est bien dans la liste
A20130925-81972 est bien dans la liste
A20130925-82050 est bien dans la liste
A20130925-82051 est bien dans la liste
A20130925-82062 est bien dans la liste
A20130925-82063 est bien dans la liste
A20130925-82064 est bien dans la liste
A20130925-82065 est bien dans la liste
A20130925-82066 est bien dans la liste
A20130925-82067 est bien dans la liste
A20130925-82136 est bien dans la liste
A20130925-82137 est bien dans la liste
A20130925-82150 est bien 

A20130927-84943 est bien dans la liste
A20130927-84965 est bien dans la liste
A20130927-84966 est bien dans la liste
A20130927-84969 est bien dans la liste
A20130927-84992 est bien dans la liste
A20130927-84993 est bien dans la liste
A20130927-85031 est bien dans la liste
A20130927-85097 est bien dans la liste
A20130927-85098 est bien dans la liste
A20130927-85144 est bien dans la liste
A20130927-85145 est bien dans la liste
A20130927-85179 est bien dans la liste
A20130927-85180 est bien dans la liste
A20130927-85222 est bien dans la liste
A20130927-85223 est bien dans la liste
A20130927-85270 est bien dans la liste
A20130927-85271 est bien dans la liste
A20130927-85272 est bien dans la liste
A20130927-85334 est bien dans la liste
A20130928-85359 est bien dans la liste
A20130928-85360 est bien dans la liste
A20130928-85388 est bien dans la liste
A20130928-85389 est bien dans la liste
A20130928-85405 est bien dans la liste
A20130928-85424 est bien dans la liste
A20130928-85457 est bien 